## Generate Static Graphs

These are the input parameters for the notebook. They will be automatically changed when the scripts to generate monthly statistics are run. You can modify them manually to generate multiple plots locally as well.

Pass in `None` to remove the filters and plot all data. This is not recommended for production settings, but might be useful for reports based on data snapshots.

In [ ]:
year = 2020
month = 11
program = "default"
study_type = "study"
mode_of_interest = None
include_test_users = False
labels = {}
use_imperial = False
sensed_algo_prefix = "cleaned"

In [ ]:
from collections import defaultdict
import datetime

import numpy as np
import pandas as pd

from plots import *
import scaffolding

import emcommon.util as emcu

sns.set_style("whitegrid")
sns.set()
%matplotlib inline

In [ ]:
# get metric vs imperial vars
label_units, dist_unit, label_units_lower, distance_col, weight_unit = scaffolding.get_units(use_imperial)

## Collect Data From Database

In [ ]:
expanded_ct, file_suffix, quality_text, debug_df = await scaffolding.load_viz_notebook_data(year,
                                                                            month,
                                                                            program,
                                                                            study_type,
                                                                            labels,
                                                                            include_test_users=include_test_users,
                                                                            add_footprint=True)
expanded_ct = scaffolding.unpack_energy_emissions(expanded_ct, use_imperial) if "mode_confirm_footprint" in expanded_ct.columns else expanded_ct

expanded_ct_sensed, file_suffix_sensed, quality_text_sensed, debug_df_sensed = await scaffolding.load_viz_notebook_sensor_inference_data(year,
                                                                            month,
                                                                            program,
                                                                            labels,
                                                                            include_test_users,
                                                                            sensed_algo_prefix)

## Data Preprocessing

In [ ]:
# Get timestamp from known year/month/day aggregated to days
sel_cols_no_label_dep = ['user_id','start_local_dt_year','start_local_dt_month','start_local_dt_day', distance_col]
sel_cols_with_label_dep = sel_cols_no_label_dep + ['mode_confirm_w_other','mode_confirm_EI','mode_confirm_CO2']

if len(expanded_ct) == 0:
    data = expanded_ct.copy()
elif "mode_confirm_w_other" not in expanded_ct.columns:
    data = expanded_ct[sel_cols_no_label_dep].copy()
else:
    #add mode confirm related columns if labeled
    data = expanded_ct[sel_cols_with_label_dep].copy()

if len(expanded_ct) > 0:
    data.rename(columns={'start_local_dt_year':'year','start_local_dt_month':'month','start_local_dt_day':'day'}, inplace=True)
    data['date_time'] = pd.to_datetime(data[['year','month','day']])
    data = data.drop(columns=['year','month','day'])
    # Categorical type will include all days/modes in groupby even if there is no data for a particular tabulation
    data.user_id = pd.Categorical(data.user_id)
    data.date_time = pd.Categorical(data.date_time)
    
if len(expanded_ct_sensed) > 0:
    expanded_ct_sensed.rename(columns={'start_local_dt_year':'year','start_local_dt_month':'month','start_local_dt_day':'day'}, inplace=True)
    
    expanded_ct_sensed['date_time'] = pd.to_datetime(expanded_ct_sensed[['year','month','day']])
    expanded_ct_sensed = expanded_ct_sensed.drop(columns=['year','month','day'])
    # Categorical type will include all days/modes in groupby even if there is no data for a particular tabulation
    expanded_ct_sensed.user_id = pd.Categorical(expanded_ct_sensed.user_id)
    expanded_ct_sensed.date_time = pd.Categorical(expanded_ct_sensed.date_time)

In [ ]:
def compute_daily_metrics(data, distance_unit):
    # Sum daily distance traveled for each mode
    mode_distance = data.groupby(['user_id','date_time','mode_confirm_w_other'], as_index=False)[[distance_unit]].sum()
    mode_distance.rename(columns={'sum':distance_unit}, inplace=True)
    mode_distance[distance_unit] = mode_distance[distance_unit].fillna(0)
    
    # Sum daily emissions for each user
    emissions = data.groupby(['user_id','date_time'], as_index=False)[[f'mode_confirm_CO2', distance_unit]].sum()
    emissions[f'mode_confirm_CO2'] = emissions[f'mode_confirm_CO2'].fillna(0)
    emissions[distance_unit] = emissions[f'mode_confirm_CO2'].fillna(0)
    
    # Sum daily energy for each user
    energy = data.groupby(['user_id','date_time'], as_index=False)[['mode_confirm_EI', distance_unit]].sum()
    energy['mode_confirm_EI'] = energy['mode_confirm_EI'].fillna(0)
    energy[distance_unit] = energy['mode_confirm_EI'].fillna(0)
    
    # Add 7-day rolling avg smoothing to better see trends
    mode_counts['trip_count_smooth'] = mode_counts.groupby(['user_id','mode_confirm_w_other'])['trip_count'].apply(lambda x: x.rolling(7,1).mean())
    mode_distance[f'{distance_unit}_smooth'] = mode_distance.groupby(['user_id','mode_confirm_w_other'])[distance_unit].apply(lambda x: x.rolling(7,1).mean())
    emissions[f'{distance_unit}_smooth'] = emissions.groupby(['user_id'])[distance_unit].apply(lambda x: x.rolling(7,1).mean())
    energy[f'{distance_unit}_smooth'] = energy.groupby(['user_id'])[distance_unit].apply(lambda x: x.rolling(7,1).mean())

    return mode_counts, mode_distance, emissions, energy

if len(expanded_ct) > 0:
    # Get the count of unique users that were active on each given date
    active_users = pd.DataFrame(data.groupby(['date_time'], as_index=False)['user_id'].nunique())
    active_users.rename(columns={'user_id':'active_users'}, inplace=True)

    if "mode_confirm_w_other" in expanded_ct.columns:
        # Count the number of trips for each confirmed mode
        mode_counts = data.groupby(['user_id','date_time','mode_confirm_w_other'], as_index=False).size()
        mode_counts.rename(columns={'size':'trip_count'}, inplace=True)
        mode_counts, mode_distance, emissions, energy = compute_daily_metrics( data, distance_unit = distance_col)

## Generate Timeseries Plots

### Emissions per week

In [ ]:
plot_title_no_quality = 'Net Daily Emissions (All Users, excluding air)'
file_name = "ts_emissions_user%s"%file_suffix

def plot_emission_per_week(emissions, unit, active_users):
    # Emissions per week across all users (net impact)
    plot_data = emissions.groupby(['date_time'], as_index=False)[f'mode_confirm_CO2'].agg(['sum'])
    total_sum = plot_data['sum'].sum()
    plot_data = plot_data.merge(active_users, on='date_time')
    plot_data['sum'] = plot_data['sum'] / plot_data['active_users']

    plot_title= plot_title_no_quality+"\n"+quality_text
    ylab = f'Emissions ({unit} CO2/day/user)'
    timeseries_plot(plot_data['date_time'], plot_data['sum'], plot_title, ylab, file_name)
    alt_text = store_alt_text_timeseries(plot_data, file_name, plot_title)

try:
    plot_emission_per_week(emissions, weight_unit, active_users)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)

### Energy per week

In [ ]:
plot_title_no_quality = 'Net Daily Energy (All Users, excluding air)'
file_name = "ts_energy_user%s"%file_suffix

try:
    # Energy per week across all users (net impact)
    plot_data = energy.groupby(['date_time'], as_index=False)['mode_confirm_EI'].agg(['sum'])
    plot_data = plot_data.merge(active_users, on='date_time')
    plot_data['sum'] = plot_data['sum'] / plot_data['active_users']

    plot_title= plot_title_no_quality+"\n"+quality_text
    ylab = 'Energy (kWH/day/user)'
    timeseries_plot(plot_data['date_time'], plot_data['sum'], plot_title, ylab, file_name)
    alt_text = store_alt_text_timeseries(plot_data, file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)

### Emissions per mile/kilometer per day

In [ ]:
plot_title_no_quality = 'Average Daily Emission Rate (All Users, excluding air)'
file_name = "ts_emissions_vmt%s"%file_suffix

def plot_emissions_per_distance_day(emissions, weight_unit, distance_unit, distance_unit_smooth,):
    # Emissions per kilometer per day across all users (travel efficiency)
    # Note that the energy plot will be identical to this one since scale factor is divided out
    emissions[f'CO2_per_{distance_unit}'] = emissions[f'mode_confirm_CO2'] / emissions[f'{distance_unit_smooth}']
    emissions[f'CO2_per_{distance_unit}'] = emissions[f'CO2_per_{distance_unit}'].fillna(0)
    plot_data = emissions.groupby(['date_time'])[f'CO2_per_{distance_unit}'].agg(['mean']).reset_index()
    
    plot_title= plot_title_no_quality+"\n"+quality_text
    ylab = f'Emissions ({weight_unit} CO2/{distance_unit}/day)'
    timeseries_plot(plot_data['date_time'], plot_data['mean'], plot_title, ylab, file_name)
    alt_text = store_alt_text_timeseries(plot_data, file_name, plot_title)

try:
    plot_emissions_per_distance_day(emissions, weight_unit, dist_unit, f'{distance_col}_smooth')

except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)

### Number of active users

In [ ]:
plot_title_no_quality = 'Number of Active Users'
file_name = "ts_users%s"%file_suffix

try:
    # Plot of active users
    plot_data = active_users

    plot_title= plot_title_no_quality+"\n"+quality_text
    ylab = 'Unique IDs'
    timeseries_plot(plot_data['date_time'], plot_data['active_users'], plot_title, ylab, file_name)
    alt_text = store_alt_text_timeseries(plot_data, file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality) 

### Daily Mode share

In [ ]:
plot_title_no_quality = 'Daily Aggregate Mode Share (excluding "Other" and "Not a trip)"'
file_name = "ts_all_modes%s"%file_suffix

values_to_translations, value_to_translations_purpose, values_to_translations_replaced = scaffolding.translate_values_to_labels(labels)

try:
    # Plot of mode share proportions across all users
    # Consolidate modes
    plot_data = mode_counts.replace('Bikeshare', 'Shared Micromobility')
    plot_data = plot_data.replace('Scooter share', 'Shared Micromobility')
    plot_data = plot_data.replace('Regular Bike', 'Personal Micromobility')
    plot_data = plot_data.replace('Skate board', 'Personal Micromobility')
    plot_data = plot_data.replace('Train', 'Transit')
    plot_data = plot_data.replace('Free Shuttle', 'Transit')
    plot_data = plot_data.replace('Bus', 'Transit')
    plot_data = plot_data.replace('Walk', 'Walk')
    plot_data = plot_data.replace('Taxi/Uber/Lyft', 'Ridehail')
    plot_data = plot_data.replace('Pilot ebike', 'E-Bike')

    plot_data = plot_data.groupby(['date_time','mode_confirm_w_other'], as_index=False)['trip_count_smooth'].sum()
    total_trips = plot_data.groupby(['date_time'], as_index=False).sum()
    plot_data = plot_data.merge(total_trips, on='date_time')
    plot_data['trip_proportion'] = plot_data['trip_count_smooth_x'] / plot_data['trip_count_smooth_y']
    # Re-establish categorical variable to not include Other and Non-trips
    plot_data = plot_data[~plot_data['mode_confirm_w_other'].isin(['not_a_trip','other'])]
    plot_data.mode_confirm_w_other = pd.Categorical(plot_data.mode_confirm_w_other, ordered=True, categories=np.unique(list(values_to_translations.keys())))    

    plot_title= plot_title_no_quality+"\n"+quality_text
    ylab = 'Proportion of All Trips'
    legend_title = 'Confirmed Mode'
    timeseries_multi_plot(plot_data, 'date_time','trip_proportion','mode_confirm_w_other', plot_title, ylab, legend_title, file_name, values_to_translations)
    alt_text = store_alt_text_generic('multivariate timeseries', file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality) 

### Sensed vs Labeled over Time

In [ ]:
plot_title_no_quality = 'Total Trips vs Labeled Trips'
file_name = "ts_totalvlabeled%s"%file_suffix

try:
    labeled_trips = data.copy()
    labeled_trips = labeled_trips.groupby(['user_id','date_time'], as_index=False).size()
    labeled_trips.rename(columns={'size':'trip_count'}, inplace=True)
    labeled_trips['trip_count_smooth'] = labeled_trips.groupby(['user_id'])['trip_count'].apply(lambda x: x.rolling(7,1).mean())
    labeled_trips.groupby(['date_time'], as_index=False).sum()
    labeled_trips = labeled_trips.groupby(['date_time'], as_index=False).sum()
    labeled_trips['Trip_type'] = "Labeled"

    total_trips = expanded_ct_sensed.copy()
    total_trips = total_trips.groupby(['user_id','date_time'], as_index=False).size()
    total_trips.rename(columns={'size':'trip_count'}, inplace=True)
    total_trips['trip_count_smooth'] = total_trips.groupby(['user_id'])['trip_count'].apply(lambda x: x.rolling(7,1).mean())
    total_trips.groupby(['date_time'], as_index=False).sum()
    total_trips = total_trips.groupby(['date_time'], as_index=False).sum()
    total_trips['Trip_type'] = "Total"

    plot_data = pd.concat([labeled_trips, total_trips])
    
    print(plot_data.head())
    
    plot_title= plot_title_no_quality+"\n"+quality_text
    ylab = 'Trip Count'
    legend_title = 'Trip Type'
    timeseries_multi_plot(plot_data, 'date_time','trip_count_smooth','Trip_type', plot_title, ylab, legend_title, file_name, values_to_translations)
    alt_text = store_alt_text_generic('multivariate timeseries', file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality) 